In [ ]:
import numpy as np
from PIL import Image
from scipy.optimize import curve_fit  #Takes the data and a given function with unknown vairables as inout and return the variables by fitting the data with the function
import matplotlib.pyplot as plt

In [ ]:
im = Image.open('pic.tif')
h,w = np.shape(im) #convert image to an array
# make a 3 dimentional array containing pixel values at (x,y) and frame index
tiffarray = np.zeros((h,w,im.n_frames))
for i in range(im.n_frames):
   im.seek(i)
   tiffarray[:,:,i] = np.array(im)
expim = tiffarray.astype(np.double);
imcut = expim[0:300,0:300,:]

In [ ]:
plt.imshow(imcut[:,:,52], cmap ="Greens", alpha = 0.7, 
           interpolation ='bilinear')

In [ ]:
# The intensity of fluorescent dyes follows a Gaussian distribution. We fit the experimental distribution to this function to find x0 which is the central position of the Gaussian function
def func(x,a,b,x0):
    return a*np.exp(-b*(x-x0)**2)

In [ ]:
x0 = []
x= np.linspace(0,300,300)

for n in range (50):
    index = np.where(imcut[:,:,n] == np.max(imcut[:,:,n]))
    x000 = []
    for i in range (index[0][0]-5,index[0][0]+5):
        #index2 = np.where(imcut[:,i,n] == np.max(imcut[:,i,n]))
        z = imcut[i,:,n]
        z = z/np.max(z)
        popt, _ = curve_fit(func, x,z)
        x000.append(popt[2])
    x0.append(np.mean(np.array(x000)))

In [ ]:
y0 = []
x= np.linspace(0,40,40)
for n in range (50):
    index = np.where(imcut[:,:,n] == np.max(imcut[:,:,n]))
    y00 = []
    for i in range (index[1][0]-5,index[1][0]+5):
        index2 = np.where(imcut[:,i,n] == np.max(imcut[:,i,n]))
        z = imcut[index2[0][0]-20:index2[0][0]+20,i,n]
        z = z/np.max(z)
        popt, _ = curve_fit(func, x,z)
        y00.append(popt[2]/2+index2[0][0])
    y0.append(np.mean(np.array(y00)))


In [ ]:
x= np.linspace(0,50,50)
plt.plot(x0,y0)
plt.gca().invert_yaxis()


In [ ]:
# convert pixels to distance
x0_positions=np.array(x0)*0.16
y0_positions=np.array(y0)*0.16
plt.plot(x0_positions,y0_positions)
plt.gca().invert_yaxis()
plt.xlabel(r"Distance ($\mu$m)")
plt.ylabel(r"Distance ($\mu$m)")
plt.title("Trajectory of a diffusing bead")

In [ ]:
    # Calculate diffusion constant
    r = np.sqrt(x0_positions**2 + y0_positions**2)
    diff = np.diff(r) #this calculates r(t + dt) - r(t)
    diff_sq = diff**2
    MSD = np.mean(diff_sq)
    D=MSD/(4*50*2)
    print(format(D,'.1E')+ " micron^2/s")